In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests10 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000
adapt_c = False


In [5]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

# Sequence 1

In [6]:
a = 0.25
num_params=9

data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/5) )
c1,c2 = calc_c(nobs)

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=500,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.94, 5th: 4.32, 10th: 5.36, 30th: 8.07, 50th: 10.37, 70th: 13.02, 90th: 18.09, 95th: 20.77, 99th: 27.32
bonus term 0.0036486575067459907 // variance stats 0.3447217511252724 //part1 3.9810717055349727 //part2 0.0009165013284420828
-1.612846673849331 -0.6052403798113133 1.8336714004212702
-1.620143988862823 -0.6052403798113133 1.8263740854077781
---
Quantiles: 1th: 3.09, 5th: 4.36, 10th: 5.30, 30th: 7.78, 50th: 10.05, 70th: 12.71, 90th: 17.88, 95th: 20.52, 99th: 26.49
bonus term 0.0035442996895254262 // variance stats 0.35488099653042365 //part1 3.9810717055349727 //part2 0.0008902878299322535
-2.0342805565212188 -0.014815568486633136 1.8886193522514625
-2.0413691559002696 -0.014815568486633136 1.8815307528724117
---
Quantiles: 1th: 2.78, 5th: 4.09, 10th: 5.00, 30th: 7.51, 50th: 9.76, 70th: 12.28, 90th: 16.74, 95th: 19.00, 99th: 24.61
bonus term 0.0033915166525201844 // variance stats 0.37088213001255366 //part1 3.9810717055349727 //part2 0.0008519104661704243
-1.986062

In [7]:


nobs = 250
c1,c2 = calc_c(nobs)

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 2.36, 5th: 3.69, 10th: 4.60, 30th: 6.96, 50th: 9.24, 70th: 11.92, 90th: 16.90, 95th: 19.99, 99th: 26.35
bonus term 0.08760738078234243 // variance stats 0.3574550731337858 //part1 3.017088168272582 //part2 0.02903706351826688
-1.803777814991885 2.050998969420157 1.959668427676246
-1.9789925765565697 2.050998969420157 1.7844536661115613
---
Quantiles: 1th: 2.90, 5th: 4.43, 10th: 5.44, 30th: 8.23, 50th: 10.65, 70th: 13.69, 90th: 19.35, 95th: 22.55, 99th: 30.24
bonus term 0.10769095499405201 // variance stats 0.33064024841179673 //part1 3.017088168272582 //part2 0.03569367184112154
-1.791516129910845 2.4921853510738528 1.9604555740629317
-2.006898039898949 2.4921853510738528 1.7450736640748274
---
Quantiles: 1th: 2.48, 5th: 3.91, 10th: 4.88, 30th: 7.48, 50th: 9.96, 70th: 13.04, 90th: 18.99, 95th: 22.44, 99th: 29.84
bonus term 0.107542164853848 // variance stats 0.3248903775972594 //part1 3.017088168272582 //part2 0.035644356033327565
-1.6811675063107925 1.2730353427521537 

# Sequence 2

In [8]:
a = 0.25
num_params=9

data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/9) )
c1,c2 = calc_c(nobs)

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=500,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.31, 5th: 3.50, 10th: 4.28, 30th: 6.45, 50th: 8.35, 70th: 10.74, 90th: 15.11, 95th: 17.32, 99th: 22.54
bonus term 0.002541321520864712 // variance stats 0.4589973252496863 //part1 1.8468761744797573 //part2 0.0013760107775393075
-1.6070564780116219 -0.5162029538633734 2.0999005970846483
-1.6121391210533513 -0.5162029538633734 2.0948179540429193
---
Quantiles: 1th: 2.51, 5th: 3.78, 10th: 4.66, 30th: 7.21, 50th: 9.31, 70th: 11.90, 90th: 16.79, 95th: 19.79, 99th: 25.44
bonus term 0.0023853348581726885 // variance stats 0.48905436958137 //part1 1.8468761744797573 //part2 0.0012915510477277171
-1.9505780227788316 -0.24332740222239896 2.020874810810449
-1.9553486924951768 -0.24332740222239896 2.016104141094104
---
Quantiles: 1th: 2.98, 5th: 4.54, 10th: 5.65, 30th: 8.68, 50th: 11.47, 70th: 14.87, 90th: 21.92, 95th: 26.05, 99th: 35.11
bonus term 0.002848584718129239 // variance stats 0.4094192610614695 //part1 1.8468761744797573 //part2 0.0015423799156062269
-1.887397383519006

In [9]:
nobs = 250
c1,c2 = calc_c(nobs)

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 2.61, 5th: 3.99, 10th: 4.80, 30th: 7.35, 50th: 9.46, 70th: 12.11, 90th: 16.94, 95th: 19.79, 99th: 25.85
bonus term 0.0519145516049603 // variance stats 0.3705363678058773 //part1 1.8468761744797573 //part2 0.028109384008694574
-1.7799377882666092 0.40660310192091736 1.9267838430891366
-1.88376689147653 0.40660310192091736 1.8229547398792159
---
Quantiles: 1th: 3.01, 5th: 4.66, 10th: 5.68, 30th: 8.52, 50th: 11.04, 70th: 14.18, 90th: 19.66, 95th: 22.97, 99th: 30.07
bonus term 0.07858909397846835 // variance stats 0.2797733610300559 //part1 1.8468761744797573 //part2 0.042552443452580656
-1.7569336894023473 1.7819798989074724 1.9432572168296727
-1.914111877359284 1.7819798989074724 1.786079028872736
---
Quantiles: 1th: 2.40, 5th: 3.78, 10th: 4.70, 30th: 6.99, 50th: 9.09, 70th: 11.64, 90th: 16.17, 95th: 18.82, 99th: 24.40
bonus term 0.05696749309768942 // variance stats 0.3213277113441375 //part1 1.8468761744797573 //part2 0.030845323517011896
-1.6396414521838307 2.32590599

# a  = .25, k= 9

In [10]:
a = 0.25
num_params=9
data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/7) )


In [11]:
nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.55, 5th: 3.85, 10th: 4.65, 30th: 7.02, 50th: 9.09, 70th: 11.70, 90th: 16.23, 95th: 18.91, 99th: 25.01
bonus term 0.06210042609976398 // variance stats 0.3535943421990102 //part1 2.2007102102809872 //part2 0.02821835687845288
-1.8597384685269684 1.5532230559524562 1.9467802026481265
-1.9839393207264964 1.5532230559524562 1.8225793504485985
---
Quantiles: 1th: 2.86, 5th: 4.29, 10th: 5.35, 30th: 7.83, 50th: 10.02, 70th: 12.75, 90th: 17.66, 95th: 20.60, 99th: 26.36
bonus term 0.05636015580884967 // variance stats 0.42504503723432824 //part1 2.2007102102809872 //part2 0.025609985151863128
-1.9569117093075314 -0.9682586351602105 1.8359453907463752
-2.0696320209252304 -0.9682586351602105 1.723225079128676
---
Quantiles: 1th: 2.83, 5th: 4.26, 10th: 5.28, 30th: 7.97, 50th: 10.35, 70th: 13.33, 90th: 18.72, 95th: 21.87, 99th: 28.99
bonus term 0.06318264375172175 // variance stats 0.40062337465927694 //part1 2.2007102102809872 //part2 0.028710115242140205
-1.839979572541174 1.179

In [12]:

nobs = 500
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.77, 5th: 4.42, 10th: 5.41, 30th: 7.95, 50th: 10.31, 70th: 13.14, 90th: 18.37, 95th: 21.03, 99th: 27.44
bonus term 0.0029963797464780556 // variance stats 0.36220078787831345 //part1 2.4297810658061283 //part2 0.0012331891908475081
-2.0733479423171524 0.5295137792161443 1.9362479221421707
-2.079340701810109 0.5295137792161443 1.9302551626492146
---
Quantiles: 1th: 3.00, 5th: 4.62, 10th: 5.61, 30th: 8.43, 50th: 10.86, 70th: 13.74, 90th: 18.86, 95th: 21.77, 99th: 27.79
bonus term 0.0029575153653034793 // variance stats 0.36696630501706556 //part1 2.4297810658061283 //part2 0.0012171941772549228
-1.937594951180885 0.632232243067286 1.761279775323427
-1.9435099819114918 0.632232243067286 1.7553647445928202
---
Quantiles: 1th: 3.21, 5th: 4.64, 10th: 5.75, 30th: 8.55, 50th: 10.92, 70th: 14.03, 90th: 19.13, 95th: 21.80, 99th: 28.97
bonus term 0.002326509919273154 // variance stats 0.4666176880905254 //part1 2.4297810658061283 //part2 0.0009574977564907842
-2.0750011054487545 

In [13]:
nobs = 100
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.10, 5th: 3.30, 10th: 4.08, 30th: 6.55, 50th: 8.84, 70th: 11.92, 90th: 18.64, 95th: 23.05, 99th: 32.91
bonus term 0.07863747097287156 // variance stats 0.4389896536268044 //part1 1.93069772888325 //part2 0.040730078974276766
-2.0242044815943645 -0.2672980516676893 1.7924767238809867
-2.1814794235401074 -0.2672980516676893 1.6352017819352433
---
Quantiles: 1th: 2.36, 5th: 3.55, 10th: 4.42, 30th: 6.90, 50th: 9.19, 70th: 11.99, 90th: 17.18, 95th: 20.57, 99th: 27.96
bonus term 0.09729242924756465 // variance stats 0.3237321743881105 //part1 1.93069772888325 //part2 0.050392367376865525
-1.726509795017481 -0.3214037979991491 1.9204293558896834
-1.92109465351261 -0.3214037979991491 1.725844497394554
---
Quantiles: 1th: 2.39, 5th: 3.88, 10th: 4.81, 30th: 7.09, 50th: 9.24, 70th: 11.91, 90th: 17.08, 95th: 19.90, 99th: 26.58
bonus term 0.06445130179457674 // variance stats 0.49466634968970513 //part1 1.93069772888325 //part2 0.03338238856884994
-1.8925910394557202 -2.09727952836

# a  = .25, k= 4, something not right?

In [14]:
a = 0.25
num_params=4



In [15]:

nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.97 & 0.98 & 0.98 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.00 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.02 & 0.02 & 0.02 & 0.01 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.972, 0.006, 0.022]), array([0.972, 0.006, 0.022]), array([0.972, 0.004, 0.024]), array([0.976, 0.014, 0.01 ]), array([0.978, 0.014, 0.008]), array([0.986, 0.012, 0.002]), -0.7745817002175927, 5.501986816440026, 5.55537286272439)


# a  = .25, K = 19

In [16]:
a = 0.25
num_params=19


In [17]:

nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.73 & 0.83 & 0.75 & 0.98 & 0.99 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.27 & 0.17 & 0.25 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.734, 0.   , 0.266]), array([0.826, 0.   , 0.174]), array([0.746, 0.   , 0.254]), array([0.982, 0.01 , 0.008]), array([0.986, 0.006, 0.008]), array([0.98, 0.01, 0.01]), -9.484346581790673, 6.130809357229824, 6.7377522665296805)


# evidence of power

In [18]:
num_params= 9
nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.83 & 0.84 & 0.84 & 0.52 & 0.54 & 0.54   \\
Model 1 & 0.17 & 0.16 & 0.16 & 0.48 & 0.46 & 0.46   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.826, 0.174, 0.   ]), array([0.836, 0.164, 0.   ]), array([0.84, 0.16, 0.  ]), array([0.52, 0.48, 0.  ]), array([0.54, 0.46, 0.  ]), array([0.538, 0.462, 0.   ]), 6.815738834996726, 4.951367809746303, 5.378514778452679)


In [19]:
nobs = 500
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.87 & 0.87 & 0.87 & 0.54 & 0.57 & 0.58   \\
Model 1 & 0.13 & 0.13 & 0.13 & 0.46 & 0.43 & 0.42   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.87, 0.13, 0.  ]), array([0.872, 0.128, 0.   ]), array([0.874, 0.126, 0.   ]), array([0.544, 0.456, 0.   ]), array([0.574, 0.426, 0.   ]), array([0.578, 0.422, 0.   ]), 6.3081574580731266, 4.960853726618843, 5.369473508225653)


In [20]:

nobs = 100
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.82 & 0.86 & 0.87 & 0.58 & 0.59 & 0.58   \\
Model 1 & 0.18 & 0.14 & 0.13 & 0.42 & 0.41 & 0.42   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.818, 0.18 , 0.002]), array([0.858, 0.14 , 0.002]), array([0.874, 0.126, 0.   ]), array([0.576, 0.424, 0.   ]), array([0.594, 0.406, 0.   ]), array([0.576, 0.424, 0.   ]), 6.149855505436059, 5.10142103370515, 5.105098914317464)


In [21]:
num_params= 19
nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.98 & 0.97 & 0.67 & 0.71 & 0.68   \\
Model 1 & 0.02 & 0.01 & 0.02 & 0.33 & 0.29 & 0.32   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.978, 0.018, 0.004]), array([0.984, 0.012, 0.004]), array([0.974, 0.022, 0.004]), array([0.666, 0.334, 0.   ]), array([0.714, 0.286, 0.   ]), array([0.676, 0.324, 0.   ]), 1.3611808889473662, 5.372628557390269, 6.1876584515809085)


In [22]:
num_params= 4
nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.59 & 0.59 & 0.62 & 0.44 & 0.48 & 0.54   \\
Model 1 & 0.41 & 0.41 & 0.38 & 0.56 & 0.52 & 0.46   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.586, 0.414, 0.   ]), array([0.586, 0.414, 0.   ]), array([0.618, 0.382, 0.   ]), array([0.44, 0.56, 0.  ]), array([0.478, 0.522, 0.   ]), array([0.54, 0.46, 0.  ]), 9.215501616836844, 4.689731641971167, 4.950275840219121)


# evidence of power 2

In [23]:
num_params= 9
nobs = 250
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.08 & 0.19 & 0.12 & 0.51 & 0.55 & 0.55   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.92 & 0.81 & 0.88 & 0.49 & 0.45 & 0.45   \\
\hline
\end{tabular}
(array([0.082, 0.   , 0.918]), array([0.19, 0.  , 0.81]), array([0.118, 0.   , 0.882]), array([0.51, 0.  , 0.49]), array([0.552, 0.   , 0.448]), array([0.554, 0.   , 0.446]), -15.091614662111798, 5.185592651718442, 5.354842222123266)


In [24]:
nobs = 500
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.08 & 0.17 & 0.11 & 0.53 & 0.60 & 0.59   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.92 & 0.83 & 0.89 & 0.47 & 0.40 & 0.41   \\
\hline
\end{tabular}
(array([0.078, 0.   , 0.922]), array([0.172, 0.   , 0.828]), array([0.106, 0.   , 0.894]), array([0.534, 0.   , 0.466]), array([0.598, 0.   , 0.402]), array([0.592, 0.   , 0.408]), -15.156145485902869, 5.306289786265359, 5.494487068787168)


In [25]:
nobs = 100
c1,c2 = calc_c(nobs)



a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.05 & 0.44 & 0.16 & 0.47 & 0.50 & 0.53   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.95 & 0.56 & 0.84 & 0.53 & 0.50 & 0.47   \\
\hline
\end{tabular}
(array([0.054, 0.   , 0.946]), array([0.436, 0.   , 0.564]), array([0.162, 0.   , 0.838]), array([0.472, 0.   , 0.528]), array([0.5, 0. , 0.5]), array([0.532, 0.   , 0.468]), -15.145785303280856, 4.73563105669253, 5.071124842368773)


In [26]:
num_params= 19
nobs = 250
c1,c2 = calc_c(nobs)


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.02 & 0.58 & 0.02 & 0.61 & 0.69 & 0.62   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.98 & 0.42 & 0.98 & 0.39 & 0.31 & 0.38   \\
\hline
\end{tabular}
(array([0.016, 0.   , 0.984]), array([0.58, 0.  , 0.42]), array([0.022, 0.   , 0.978]), array([0.612, 0.   , 0.388]), array([0.692, 0.   , 0.308]), array([0.624, 0.   , 0.376]), -20.428208240439005, 5.484469676637277, 6.186565495715466)


In [27]:
num_params= 4
nobs = 250
c1,c2 = calc_c(nobs)


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.21 & 0.22 & 0.26 & 0.44 & 0.47 & 0.56   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.79 & 0.78 & 0.74 & 0.56 & 0.53 & 0.44   \\
\hline
\end{tabular}
(array([0.212, 0.   , 0.788]), array([0.218, 0.   , 0.782]), array([0.262, 0.   , 0.738]), array([0.442, 0.   , 0.558]), array([0.474, 0.   , 0.526]), array([0.558, 0.   , 0.442]), -12.162401508818723, 4.59784076950117, 4.884784663363507)
